In [525]:
import numpy as np
import pandas as pd
import sklearn as sk
from scipy import sparse
from itertools import product

# Read CSV

In [526]:
df=pd.read_csv('zoo.csv')

# Zad 1

In [527]:

def freq(x, prob=True):
    
    if sparse.issparse(x):
        #input is scipy.sparse
        pass
         
    else:
        #input is Series
        tmp=x.value_counts(normalize=prob)
    
    
    return tmp


freq(df['feathers'])

False    0.80198
True     0.19802
Name: feathers, dtype: float64

# Zad 2

In [664]:
def freq2(x, y, prob=True):
    if sparse.issparse(x) and sparse.issparse(y):  
        #input is scipy.sparse
        index = pd.MultiIndex.from_product([np.unique(np.append(x.data,0)),np.unique(np.append(y.data,0))],names=['col1','col2'])
        tmp=pd.Series(index=index,dtype=np.float64)


        

        for val in index:
            mask=np.logical_and((x==1).toarray(),(y==1).toarray())
            tmp[val]=sum(mask)

        if prob:
            tmp=tmp/sum(tmp)
            
        return tmp
        
    else:
        #input is Series
        
        df=pd.DataFrame({x.name:x,y.name:y})
        
        index = pd.MultiIndex.from_product([np.unique(x),np.unique(y)],names=[x.name,y.name])
        tmp=pd.Series(index=index,dtype=np.float64)
        
        for val in index:
            tmp[val]=sum(df.eq(val).all(axis=1))

        if prob:
            tmp=tmp/sum(tmp)
    
    
        
    return tmp

freq2(sparse.coo_array(df['feathers']),sparse.coo_array(df['legs']))


C:\Users\zubi1\AppData\Local\Temp\ipykernel_17356\3710791860.py:11: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  mask=np.logical_and((x==0).toarray(),(y==0).toarray())


# Zad 3

In [529]:
def entropy(p):
    return np.sum(p*np.log2(1/p))


def infogain(x,y):
    xp=freq(x)
    yp=freq(y)
    xyp=freq2(x,y)
    
    return entropy(xp) + entropy(yp) - entropy(xyp)

    
import scipy.stats._entropy

#TEST
print('Numpy entropy:\t',scipy.stats._entropy.entropy(freq(df['feathers']),base=2))
print('My entropy:\t',entropy( freq(df['feathers'])) )

print('Infogain:\t',infogain(df['feathers'],df['tail']))

Numpy entropy:	 0.7179499765002912
My entropy:	 0.7179499765002912
Infogain:	 0.09668209882120649


# Zad 4

In [530]:
indexes=pd.MultiIndex.from_product([df.columns[1:-1],['type']])

infos=pd.Series(index=indexes,dtype=np.float64,name='info gains')

for cols in indexes:
    infos.loc[cols]=infogain(df[cols[0]],df[cols[1]])

infos.sort_values(ascending=False,inplace=True)
infos

    

legs      type    1.363047
milk      type    0.974320
toothed   type    0.865694
eggs      type    0.830138
hair      type    0.790675
feathers  type    0.717950
backbone  type    0.676163
breathes  type    0.614494
tail      type    0.500460
airborne  type    0.469703
fins      type    0.466614
aquatic   type    0.389487
catsize   type    0.308490
venomous  type    0.133090
predator  type    0.093447
domestic  type    0.050669
Name: info gains, dtype: float64

# Zad 5

In [531]:
from sklearn.datasets import fetch_rcv1
rcv1=fetch_rcv1(subset='train')

### maska wierszy

In [532]:
atrybut='GSPO'
tindex=(rcv1.target_names==atrybut).nonzero()[0]
rows=rcv1.target[:,tindex].nonzero()[0]

### binaryzacja

In [533]:
data_bin=rcv1.data
mask=data_bin.nonzero()
data_bin[mask[0],mask[1]]=1
print(data_bin)

  (0, 863)	1.0
  (0, 1522)	1.0
  (0, 1680)	1.0
  (0, 2292)	1.0
  (0, 2844)	1.0
  (0, 2866)	1.0
  (0, 3239)	1.0
  (0, 4124)	1.0
  (0, 4270)	1.0
  (0, 4664)	1.0
  (0, 5215)	1.0
  (0, 5572)	1.0
  (0, 5698)	1.0
  (0, 5793)	1.0
  (0, 6221)	1.0
  (0, 6591)	1.0
  (0, 7226)	1.0
  (0, 7974)	1.0
  (0, 8144)	1.0
  (0, 8758)	1.0
  (0, 8770)	1.0
  (0, 8900)	1.0
  (0, 8926)	1.0
  (0, 8939)	1.0
  (0, 9106)	1.0
  :	:
  (23148, 35121)	1.0
  (23148, 35622)	1.0
  (23148, 35781)	1.0
  (23148, 36494)	1.0
  (23148, 36637)	1.0
  (23148, 36693)	1.0
  (23148, 37372)	1.0
  (23148, 38892)	1.0
  (23148, 39195)	1.0
  (23148, 39524)	1.0
  (23148, 39912)	1.0
  (23148, 39924)	1.0
  (23148, 40047)	1.0
  (23148, 40335)	1.0
  (23148, 42103)	1.0
  (23148, 42520)	1.0
  (23148, 43436)	1.0
  (23148, 43624)	1.0
  (23148, 44229)	1.0
  (23148, 44946)	1.0
  (23148, 45050)	1.0
  (23148, 45500)	1.0
  (23148, 45791)	1.0
  (23148, 46158)	1.0
  (23148, 46199)	1.0


### Infogain

In [540]:
reference=rows[0]

indexes_to_calc=list(product( np.arange(data_bin.shape[0]), [reference] ))

infos=pd.Series(index=pd.MultiIndex.from_tuples(indexes_to_calc),dtype=np.float64,name='info gains')

for index in indexes_to_calc[:10]:
    #infos.loc[index]=infogain(data_bin[index[0],:],data_bin[index[1],:])
    pass
    